# Chapter 16: Portfolio Construction
AFML focus: convert signal insights into diversified allocations.


In [1]:
import math
import matplotlib.pyplot as plt
import polars as pl
import openquant
import sys
from pathlib import Path
sys.path.insert(0, str(Path('notebooks/python/scripts').resolve()))
from afml_chapter_utils import (
    fetch_panel,
    simple_returns,
    probs_and_sides_from_momentum,
    timestamps_from_dates,
    lag_corr,
    fracdiff_ffd,
)

panel = fetch_panel(window=900)
dates = panel['date'].to_list()
uso = panel['USO'].to_list()
uso_ret = simple_returns(uso)
probs, sides = probs_and_sides_from_momentum(uso)
timestamps = timestamps_from_dates(dates)
asset_names = ['USO', 'BNO', 'XLE', 'GLD', 'UNG']
asset_prices = panel.select(asset_names).rows()
print('rows', panel.height, 'range', dates[0], dates[-1])


rows 900 range 2022-07-08 2026-02-06


In [2]:
ivp = openquant.portfolio.allocate_inverse_variance(asset_prices)
mv = openquant.portfolio.allocate_min_vol(asset_prices)
msr = openquant.portfolio.allocate_max_sharpe(asset_prices, risk_free_rate=0.0)

w = pl.DataFrame({
    'asset': asset_names,
    'ivp': ivp[0],
    'min_vol': mv[0],
    'max_sharpe': msr[0],
})

plt.figure(figsize=(10,4))
for c in ['ivp','min_vol','max_sharpe']:
    plt.plot(w['asset'], w[c], marker='o', label=c)
plt.title('Chapter 16: Allocation Comparison')
plt.legend()
plt.tight_layout()
plt.show()

print({'ivp_sharpe': ivp[3], 'mv_sharpe': mv[3], 'msr_sharpe': msr[3]})


{'ivp_sharpe': 0.0, 'mv_sharpe': 0.0, 'msr_sharpe': -0.2789077637869617}


## Interpretation
Different allocation objectives produce materially different risk/return surfaces; strategy evaluation must include allocation choice, not only signal quality.

